In [1]:
%matplotlib widget

In [2]:
from pathlib import Path
from split_dataset import SplitDataset
from skimage import io
import flammkuchen as fl
import numpy as np
from scipy.stats import zscore
from bouter.utilities import fast_corrcoef
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpecFromSubplotSpec
from lotr.plotting import add_cbar

In [3]:
path = Path(r'C:\Users\You\Documents\test')

aligned_ds = SplitDataset(path/"rotated")
mask_all = io.imread(path/"mask_rotated.tif", plugin="tifffile")
fish_plane = fl.load(path/"fish_plane.h5")

In [4]:
bin_px = 10

pooled = []
x_pooled = []
y_pooled = []

for plane in range(aligned_ds.shape[1]):
    aligned = aligned_ds[:,plane,:,:]
    mask = mask_all[plane,:,:]
    
    px_mask = np.argwhere(mask>=1)
    px_str = [str(x0)+str(y0) for x0, y0 in px_mask]
    x_loc = np.unique(px_mask[:,0])
    y_loc = np.unique(px_mask[:,1])
    x_bins = np.digitize(x_loc, np.arange(x_loc.min(), x_loc.max()+1, bin_px)) - 1
    y_bins = np.digitize(y_loc, np.arange(y_loc.min(), y_loc.max()+1, bin_px)) - 1
    n_bin_x = x_bins.max() + 1
    n_bin_y = y_bins.max() + 1
    
    traces = np.full((aligned.shape[0], n_bin_x*n_bin_y), np.nan)
    for x in range(n_bin_x):
        for y in range(n_bin_y):
            px_here = np.array([(x0, y0) for x0 in x_loc[x_bins==x] for y0 in y_loc[y_bins==y]])
            px_here_str = [str(x0)+str(y0) for x0, y0 in px_here]
            px_here = px_here[np.isin(px_here_str, px_str)]
            if len(px_here) > 0:
                traces[:, x*n_bin_y+y] = np.nanmean(aligned[:, px_here[:,0], px_here[:,1]], axis=-1)
    traces = zscore(traces, axis=0, nan_policy='omit')
    cor = fast_corrcoef(traces.T)
    mx = np.full((n_bin_x*n_bin_y, n_bin_x*2-1, n_bin_y*2-1), np.nan)
    for i in range(len(cor)):
        if not np.isnan(cor[i,np.arange(len(cor))!=i]).all():
            x0 = i//n_bin_y
            y0 = i%n_bin_y
            for j in range(len(cor)):
                x = j//n_bin_y
                y = j%n_bin_y
                mx[x0*n_bin_y+y0, x-x0+n_bin_x-1, y-y0+n_bin_y-1] = cor[i,j]
    pooled.append(np.nanmean(mx, axis=0))
    
    traces = np.full((aligned.shape[0], n_bin_x), np.nan)
    for x in range(n_bin_x):
        px_here = np.array([(x0, y0) for x0 in x_loc[x_bins==x] for y0 in y_loc])
        px_here_str = [str(x0)+str(y0) for x0, y0 in px_here]
        px_here = px_here[np.isin(px_here_str, px_str)]
        if len(px_here) > 0:
            traces[:, x] = np.nanmean(aligned[:, px_here[:,0], px_here[:,1]], axis=-1)
    traces = zscore(traces, axis=0, nan_policy='omit')
    cor = fast_corrcoef(traces.T)
    mx = np.full((n_bin_x, n_bin_x*2-1), np.nan)
    for i0 in range(len(cor)):
        for i in range(len(cor)):
            mx[i0, i-i0+n_bin_x-1] = cor[i0,i]
    x_pooled.append(np.nanmean(mx, axis=0))

    traces = np.full((aligned.shape[0], n_bin_y), np.nan)
    for y in range(n_bin_y):
        px_here = np.array([(x0, y0) for x0 in x_loc for y0 in y_loc[y_bins==y]])
        px_here_str = [str(x0)+str(y0) for x0, y0 in px_here]
        px_here = px_here[np.isin(px_here_str, px_str)]
        if len(px_here) > 0:
            traces[:, y] = np.nanmean(aligned[:, px_here[:,0], px_here[:,1]], axis=-1)
    traces = zscore(traces, axis=0, nan_policy='omit')
    cor = fast_corrcoef(traces.T)
    mx = np.full((n_bin_y, n_bin_y*2-1), np.nan)
    for i0 in range(len(cor)):
        for i in range(len(cor)):
            mx[i0, i-i0+n_bin_y-1] = cor[i0,i]
    y_pooled.append(np.nanmean(mx, axis=0))

C:\Users\You\miniconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: Mean of empty slice


In [5]:
px_size = 0.35

fig = plt.figure(figsize=(7, 7))
gs = fig.add_gridspec(4, 3, wspace=0.5, hspace=0.5)

for i in range(len(pooled)):
    subgs = GridSpecFromSubplotSpec(2, 2, gs[i], width_ratios=[10,2], height_ratios=[2,10], wspace=0, hspace=0)
    
    ax = fig.add_subplot(subgs[1,0])
    mx = pooled[i]
    x_size, y_size = mx.shape
    x_ext = (x_size//2)*bin_px*px_size
    y_ext = (y_size//2)*bin_px*px_size
    lim = np.ceil(np.maximum(np.nanmax(mx[mx!=1]), abs(np.nanmin(mx)))*10)/10
    im = ax.imshow(mx.T, vmin=-lim, vmax=lim, extent=(-x_ext, x_ext, -y_ext, y_ext), cmap='RdBu_r', origin="upper")
    ax.xaxis.set(ticks_position='bottom')
    ax.tick_params(labelsize=6)
    ax.set(aspect="equal", xticks=(-50, 0, 50), yticks=(-25, 0, 25))
    if i == 0:
        ax.set(xlabel="frontal axis [μm]", ylabel="sagittal axis [μm]")
    mx_bbox = ax.get_position()
    add_cbar(im, ax, inset_loc=(1.2, -0.1, 0.05, 0.3), ticks=(-lim,lim), labelsize=6)

    ax = fig.add_subplot(subgs[0,0])
    bbox = ax.get_position()
    ax.plot(x_pooled[i], c='gray', linewidth=1)
    ax.margins(x=0)
    ax.set(position=[mx_bbox.x0, mx_bbox.y0+mx_bbox.height, mx_bbox.width, bbox.height])
    ax.set_title("f"+str(hash(fish_plane[i]))[-5:], loc="left", fontsize=6)
    ax.axis('off')

    ax = fig.add_subplot(subgs[1,1])
    bbox = ax.get_position()
    ax.plot(y_pooled[i], range(y_size), c='gray', linewidth=1)
    ax.margins(y=0)
    ax.set(position=[mx_bbox.x0+mx_bbox.width, mx_bbox.y0, bbox.width, mx_bbox.height])
    ax.axis('off')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
plt.savefig(path/"SupFig16.pdf")